In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import statsmodels.api as sm
from statsmodels.tools.eval_measures import mse, rmse

%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format

import warnings
warnings.filterwarnings('ignore')

Bu ödevde, ev fiyatları verileriyle çalışmaya devam edeceksiniz. Bu ödevi tamamlamak için, aşağıdaki görevlerin çözümlerini içeren bir Jupyter not defterini link olarak gönderin:

1) Ev Fiyatları verilerini Kaggle'dan indirin.

2) Önceki dersteki modelinizi yeniden uygulayın.

3) Aynı model tanımını kullanarak OLS, Lasso, Ridge ve ElasticNet regresyonlarını deneyin. Bu kez, modelleriniz için en iyi hiperparametre değerlerini seçmek için k-fold çapraz doğrulama yapmanız gerekir. Hangi model en iyisidir? Açıklayın?

In [2]:
ev_fiyatlari = pd.read_csv('data/train.csv')

In [3]:
y = ev_fiyatlari['SalePrice']

X = ev_fiyatlari[['YearBuilt', 'TotRmsAbvGrd', 'FullBath' , '1stFlrSF', 'TotalBsmtSF',
                  'GarageArea', 'GarageCars', 'GrLivArea', 'OverallQual', 'YearRemodAdd', 'Fireplaces',
                  'BsmtFinSF1', 'WoodDeckSF','2ndFlrSF', 'OpenPorchSF', 'HalfBath']]

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 500)

lrm = LinearRegression()
lrm.fit(X_train, y_train)

y_egitim_tahmini = lrm.predict(X_train)
y_test_tahmini = lrm.predict(X_test)

In [7]:
print("Eğitim kümesindeki gözlem sayısı : {}".format(X_train.shape[0]))
print("Test kümesindeki gözlem sayısı   : {}".format(X_test.shape[0]))

print("Eğitim kümesindeki R-Kare değeri  : {}".format(lrm.score(X_train, y_train)))
print("-----Test kümesi istatistikleri---")
print("Test kümesi R-Kare değeri         : {}".format(lrm.score(X_test, y_test)))
print("Ortalama Mutlak Hata (MAE)        : {}".format(mean_absolute_error(y_test, y_test_tahmini)))
print("Ortalama Kare Hata (MSE)          : {}".format(mse(y_test, y_test_tahmini)))
print("Kök Ortalama Kare Hata (RMSE)     : {}".format(rmse(y_test, y_test_tahmini)))
print("Ortalama Mutlak Yüzde Hata (MAPE) : {}".format(np.mean(np.abs((y_test - y_test_tahmini) / y_test)) * 100))

Eğitim kümesindeki gözlem sayısı : 1168
Test kümesindeki gözlem sayısı   : 292
Eğitim kümesindeki R-Kare değeri  : 0.7801794585413929
-----Test kümesi istatistikleri---
Test kümesi R-Kare değeri         : 0.8296187584623316
Ortalama Mutlak Hata (MAE)        : 20673.626321627264
Ortalama Kare Hata (MSE)          : 828761729.8038493
Kök Ortalama Kare Hata (RMSE)     : 28788.222067433224
Ortalama Mutlak Yüzde Hata (MAPE) : 13.042167043498004


In [8]:
x_train = sm.add_constant(X_train)

sonuclar = sm.OLS(y_train, X_train).fit()

sonuclar.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.962
Model:                            OLS   Adj. R-squared:                  0.961
Method:                 Least Squares   F-statistic:                     1806.
Date:                Sat, 20 Apr 2019   Prob (F-statistic):               0.00
Time:                        14:37:12   Log-Likelihood:                -14006.
No. Observations:                1168   AIC:                         2.804e+04
Df Residuals:                    1152   BIC:                         2.812e+04
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
YearBuilt      -31.9344     56.435     -0.566      0.572    -142.661      78.792
TotRmsAbvGrd   697.0836   1291.052      0.540      0.589   -1835.993    3230.160
FullBath      8077.9526   3146.561      2.567      0.010    1904.320    1.43e+04
1stFlrSF        32.4401     24.517      1.323      0.186     -15.664      80.544
TotalBsmtSF     12.9048      5.041      2.560      0.011       3.014      22.796
GarageArea       8.8850     12.215      0.727      0.467     -15.081      32.851
GarageCars    1.474e+04   3536.615      4.167      0.000    7797.688    2.17e+04
GrLivArea        6.3416     23.930      0.265      0.791     -40.609      53.292
OverallQual   2.436e+04   1289.261     18.896      0.000    2.18e+04    2.69e+04
YearRemodAdd   -19.4665     55.532     -0.351      0.726    -128.421      89.488
Fireplaces    5241.3160   2170.326      2.415      0.016     983.082    9499.550
BsmtFinSF1      19.0558      3.063      6.220      0.000      13.045      25.066
WoodDeckSF      40.0425      9.788      4.091      0.000      20.837      59.248
2ndFlrSF        21.7879     24.245      0.899      0.369     -25.781      69.357
OpenPorchSF     12.3433     18.670      0.661      0.509     -24.288      48.974
HalfBath      6534.1558   3221.423      2.028      0.043     213.642    1.29e+04
==============================================================================
Omnibus:                      385.037   Durbin-Watson:                   1.969
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            44354.315
Skew:                          -0.426   Prob(JB):                         0.00
Kurtosis:                      33.177   Cond. No.                     1.22e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.22e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

***Ridge fonksiyonu***

In [9]:
from sklearn.linear_model import Ridge

ridgeregr = Ridge(alpha=10**37) 
ridgeregr.fit(X_train, y_train)

y_egitim_tahmini = ridgeregr.predict(X_train)
y_test_tahmini = ridgeregr.predict(X_test)

print("Eğitim kümesi R-Kare değeri       : {}".format(ridgeregr.score(X_train, y_train)))
print("-----Test kümesi istatistikleri---")
print("Test kümesi R-Kare değeri         : {}".format(ridgeregr.score(X_test, y_test)))
print("Ortalama Mutlak Hata (MAE)        : {}".format(mean_absolute_error(y_test, y_test_tahmini)))
print("Ortalama Kare Hata (MSE)          : {}".format(mse(y_test, y_test_tahmini)))
print("Kök Ortalama Kare Hata (RMSE)     : {}".format(rmse(y_test, y_test_tahmini)))
print("Ortalama Mutlak Yüzde Hata (MAPE) : {}".format(np.mean(np.abs((y_test - y_test_tahmini) / y_test)) * 100))

Eğitim kümesi R-Kare değeri       : 0.0
-----Test kümesi istatistikleri---
Test kümesi R-Kare değeri         : -0.004908518792945626
Ortalama Mutlak Hata (MAE)        : 50966.240124788885
Ortalama Kare Hata (MSE)          : 4888036469.351241
Kök Ortalama Kare Hata (RMSE)     : 69914.49398623465
Ortalama Mutlak Yüzde Hata (MAPE) : 32.62793758351267


***Lasso regresyonu***

In [10]:
from sklearn.linear_model import Lasso

lassoregr = Lasso(alpha=10**20.5) 
lassoregr.fit(X_train, y_train)

y_egitim_tahmini = lassoregr.predict(X_train)
y_test_tahmini = lassoregr.predict(X_test)

print("Eğitim kümesi R-Kare değeri       : {}".format(lassoregr.score(X_train, y_train)))
print("-----Test kümesi istatistikleri---")
print("Test kümesi R-Kare değeri         : {}".format(lassoregr.score(X_test, y_test)))
print("Ortalama Mutlak Hata (MAE)        : {}".format(mean_absolute_error(y_test, y_test_tahmini)))
print("Ortalama Kare Hata (MSE)          : {}".format(mse(y_test, y_test_tahmini)))
print("Kök Ortalama Kare Hata (RMSE)     : {}".format(rmse(y_test, y_test_tahmini)))
print("Ortalama Mutlak Yüzde Hata (MAPE) : {}".format(np.mean(np.abs((y_test - y_test_tahmini) / y_test)) * 100))

Eğitim kümesi R-Kare değeri       : 0.0
-----Test kümesi istatistikleri---
Test kümesi R-Kare değeri         : -0.004908518792945626
Ortalama Mutlak Hata (MAE)        : 50966.240124788885
Ortalama Kare Hata (MSE)          : 4888036469.351241
Kök Ortalama Kare Hata (RMSE)     : 69914.49398623465
Ortalama Mutlak Yüzde Hata (MAPE) : 32.62793758351267


***ElasticNet regresyonu***

In [11]:
from sklearn.linear_model import ElasticNet

elasticregr = ElasticNet(alpha=10**21, l1_ratio=0.5) 
elasticregr.fit(X_train, y_train)

y_egitim_tahmini = elasticregr.predict(X_train)
y_test_tahmini = elasticregr.predict(X_test)

print("Eğitim kümesi R-Kare değeri       : {}".format(elasticregr.score(X_train, y_train)))
print("-----Test kümesi istatistikleri---")
print("Test kümesi R-Kare değeri         : {}".format(elasticregr.score(X_test, y_test)))
print("Ortalama Mutlak Hata (MAE)        : {}".format(mean_absolute_error(y_test, y_test_tahmini)))
print("Ortalama Kare Hata (MSE)          : {}".format(mse(y_test, y_test_tahmini)))
print("Kök Ortalama Kare Hata (RMSE)     : {}".format(rmse(y_test, y_test_tahmini)))
print("Ortalama Mutlak Yüzde Hata (MAPE) : {}".format(np.mean(np.abs((y_test - y_test_tahmini) / y_test)) * 100))

Eğitim kümesi R-Kare değeri       : 0.0
-----Test kümesi istatistikleri---
Test kümesi R-Kare değeri         : -0.004908518792945626
Ortalama Mutlak Hata (MAE)        : 50966.240124788885
Ortalama Kare Hata (MSE)          : 4888036469.351241
Kök Ortalama Kare Hata (RMSE)     : 69914.49398623465
Ortalama Mutlak Yüzde Hata (MAPE) : 32.62793758351267


k-fold 

In [12]:
from sklearn.model_selection import KFold

In [13]:
kf = KFold(n_splits=2)
kf.get_n_splits(X)

2

In [14]:
print(kf)

KFold(n_splits=2, random_state=None, shuffle=False)


In [17]:
KFold(n_splits=2, random_state=None, shuffle=False)

for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    

TRAIN: [ 730  731  732  733  734  735  736  737  738  739  740  741  742  743
  744  745  746  747  748  749  750  751  752  753  754  755  756  757
  758  759  760  761  762  763  764  765  766  767  768  769  770  771
  772  773  774  775  776  777  778  779  780  781  782  783  784  785
  786  787  788  789  790  791  792  793  794  795  796  797  798  799
  800  801  802  803  804  805  806  807  808  809  810  811  812  813
  814  815  816  817  818  819  820  821  822  823  824  825  826  827
  828  829  830  831  832  833  834  835  836  837  838  839  840  841
  842  843  844  845  846  847  848  849  850  851  852  853  854  855
  856  857  858  859  860  861  862  863  864  865  866  867  868  869
  870  871  872  873  874  875  876  877  878  879  880  881  882  883
  884  885  886  887  888  889  890  891  892  893  894  895  896  897
  898  899  900  901  902  903  904  905  906  907  908  909  910  911
  912  913  914  915  916  917  918  919  920  921  922  923  924  925

KeyError: '[ 730  731  732  733  734  735  736  737  738  739  740  741  742  743\n  744  745  746  747  748  749  750  751  752  753  754  755  756  757\n  758  759  760  761  762  763  764  765  766  767  768  769  770  771\n  772  773  774  775  776  777  778  779  780  781  782  783  784  785\n  786  787  788  789  790  791  792  793  794  795  796  797  798  799\n  800  801  802  803  804  805  806  807  808  809  810  811  812  813\n  814  815  816  817  818  819  820  821  822  823  824  825  826  827\n  828  829  830  831  832  833  834  835  836  837  838  839  840  841\n  842  843  844  845  846  847  848  849  850  851  852  853  854  855\n  856  857  858  859  860  861  862  863  864  865  866  867  868  869\n  870  871  872  873  874  875  876  877  878  879  880  881  882  883\n  884  885  886  887  888  889  890  891  892  893  894  895  896  897\n  898  899  900  901  902  903  904  905  906  907  908  909  910  911\n  912  913  914  915  916  917  918  919  920  921  9